# Main Notebook

In [1]:
import os
import gc
import pytz
import operator
import numpy as np
import pickle as pkl
import xgboost as xgb
from time import sleep
from datetime import datetime
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.python.keras import callbacks
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.losses import mean_absolute_error
from tensorflow.python.keras.layers import Dense, Input, Activation
from tensorflow.python.keras.layers import BatchNormalization, Add, Dropout
from tensorflow.python.keras.layers.advanced_activations import LeakyReLU
from tensorflow.python.keras.optimizers import Adam, Adadelta, SGD

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action = 'ignore', category = FutureWarning)
warnings.filterwarnings(action = 'ignore', category = DeprecationWarning)

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import sys
sys.path.append('..')
from utils.generate_features import *

In [2]:
datetime.now(pytz.timezone('Europe/Oslo')).strftime('%m.%d.%Y_%H.%M.%S')

'08.13.2019_12.24.30'

## General Methods

In [3]:
def memory_optimization(dfs):
    for df in dfs:
        del df
    gc.collect()

## Split Training and Validation

In [4]:
def split(df_train):
    train_X, validation_X = train_test_split(df_train, test_size = 0.1, random_state = 0)

    train_X = train_X.reset_index()
    validation_X = validation_X.reset_index()

    train_y = train_X['scalar_coupling_constant']
    train_y = train_y.replace([np.inf, -np.inf], np.nan)
    train_y = train_y.reset_index()
    train_y = train_y.drop(['index'], axis = 1)
    validation_y = validation_X['scalar_coupling_constant']
    validation_y = validation_y.replace([np.inf, -np.inf], np.nan)
    validation_y = validation_y.reset_index()
    validation_y = validation_y.drop(['index'], axis = 1)

    train_X = train_X.drop('scalar_coupling_constant', axis = 1)
    validation_X = validation_X.drop('scalar_coupling_constant', axis = 1)
    
    train_X = train_X.drop(['index'], axis = 1)
    validation_X = validation_X.drop(['index'], axis = 1)
    
    return train_X, train_y, validation_X, validation_y

## NN Training

In [5]:
def create_nn_model(input_shape):
    # input layer
    inp = Input(shape = (input_shape,))

    # first hidden layer
    x = Dense(256, kernel_initializer = 'he_normal')(inp)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha = 0.05)(x)
    x = Dropout(0.2)(x)
    
    # second hidden layer
    x = Dense(512, kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha = 0.05)(x)
    x = Dropout(0.2)(x)
    
    # third hidden layer
    x = Dense(1024, kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha = 0.05)(x)
    x = Dropout(0.2)(x)
    
    # fourth hidden layer
    x = Dense(512, kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha = 0.05)(x)
    x = Dropout(0.2)(x)
    
    # fifth hidden layer
    x = Dense(256, kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha = 0.05)(x)
    x = Dropout(0.2)(x)
    
    # sixth hidden layer
    x = Dense(128, kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha = 0.05)(x)
    x = Dropout(0.2)(x)
    
    # seventh hidden layer
    x = Dense(128, kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha = 0.05)(x)
    x = Dropout(0.2)(x)
    
    # eight hidden layer
    x = Dense(64, kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha = 0.05)(x)
    x = Dropout(0.2)(x)
    
    # output layer scalar_coupling_constant
    out = Dense(1, activation = 'linear')(x)   
    model = Model(inputs = inp, outputs = out)
    return model

In [6]:
def nn_train(coupling_type, train_X, train_y, validation_X, validation_y):
    epoch_n = 2000
    verbose = 1
    batch_size = 2048
    model_name = f'../models/nn/coupling_model_{coupling_type}_NN.hdf5'
    
    nn_model = create_nn_model(train_X.shape[1])

    nn_model.compile(loss = 'mae', optimizer = Adam())
    
    es = callbacks.EarlyStopping(monitor = 'loss', min_delta = 0.00001, patience = 64,
                                 verbose = verbose, mode = 'auto', restore_best_weights = True)
    
    rlr = callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 32,
                                      min_lr = 1e-7, mode = 'auto', verbose = verbose)
    
    sv_mod = callbacks.ModelCheckpoint(model_name, monitor = 'val_loss', save_best_only = True,
                                       save_weights_only = True, restore_best_weights = True)
    
    history = nn_model.fit(x = train_X.values, y = train_y['scalar_coupling_constant'].values, 
                           validation_data = (validation_X.values, validation_y['scalar_coupling_constant'].values),
                           callbacks = [es, rlr, sv_mod], epochs = epoch_n, batch_size = batch_size, verbose = verbose)
    
    nn_model.save_weights(model_name)

    return nn_model

## XGB Training

In [7]:
def xgb_train(coupling_type, train_X, train_y, validation_X, validation_y):
    model_name_wrt = f'../models/xgb/coupling_model_{coupling_type}_XGB.hdf5'

    xgb_model = xgb.XGBRegressor(base_score = 0.5, booster = 'gbtree', colsample_bylevel = 1,
                                 colsample_bytree = 1, gamma = 0, importance_type = 'gain',
                                 learning_rate = 0.1, max_delta_step = 0, max_depth = 9,
                                 min_child_weight = 1, missing = None, n_estimators = 10000, n_jobs = -1,
                                 nthread = None, objective = 'reg:squarederror', random_state = 101, reg_alpha = 2,
                                 reg_lambda = 0.2, scale_pos_weight = 1, seed = None, silent = False, subsample = 1)

    xgb_model.fit(train_X, train_y, eval_set = [(validation_X, validation_y)], eval_metric = 'mae', 
                  early_stopping_rounds = 32, verbose = True)   
    
    xgb_model.save_model(model_name_wrt)
    #joblib.dump(xgb_model, model_name_wrt)
    
    return xgb_model

In [8]:
def importance(xgb_model, train_X):
    input_features = train_X.columns.values
    feat_imp = xgb_model.feature_importances_
    np.split(feat_imp, len(input_features))
    
    feat_imp_dict = {}
    for i in range(0, len(input_features)):
        feat_imp_dict[feat_imp[i]] = input_features[i]

    sorted_feats = sorted(feat_imp_dict.items(), key = operator.itemgetter(0))
    for i in range(len(sorted_feats) - 1, 0, -1):
        print(sorted_feats[i])

## Training, Blending and Submission

In [9]:
def val_blending_list(nn_val_predict, xgb_val_predict, validation_y):
    nn_val_pred_error  = (validation_y - nn_val_predict)
    xgb_val_pred_error = (validation_y - xgb_val_predict)
    
    val_error_corr = np.corrcoef(nn_val_pred_error, xgb_val_pred_error)
    print(f'Error correlation: {val_error_corr[0][1]}. Error difference {xgb_accuracy - nn_accuracy}')
    
    log_accuracy = 0
    log_accuracy_list = []
    val_predict = np.array([])

    for alpha in np.arange(0, 1.1, 0.1):
        nn_val_predict_scaled = alpha * nn_val_predict
        xgb_val_predict_scaled = (1 - alpha) * xgb_val_predict
        val_predict = nn_val_predict_scaled + xgb_val_predict_scaled
        log_accuracy = np.log(np.mean(np.abs(validation_y - val_predict)))
        log_accuracy_list.append(log_accuracy)

    return log_accuracy_list

In [10]:
load = False

df_test = pd.read_csv('../submissions/submission_best.csv')
test_prediction = df_test['scalar_coupling_constant']
df_test_full = pd.read_csv('../input/test.csv')

start_time = datetime.now()

val_score = {
    '1JHC': np.inf, '1JHN': np.inf, '2JHH': np.inf, '2JHC': np.inf, 
    '2JHN': np.inf, '3JHH': np.inf, '3JHC': np.inf, '3JHN': np.inf
}

coupling_types = ['1JHN', '1JHC', '2JHH', '2JHC', '2JHN', '3JHH', '3JHC', '3JHN']

for coupling_type in coupling_types:
    time = datetime.now(pytz.timezone('Europe/Oslo')).strftime('%m.%d.%Y_%H.%M.%S')
    print(f'Predicting {coupling_type} out of {coupling_types} at {time}.')
    
    df_train, df_test = get_features(coupling_type)
    train_X, train_y, validation_X, validation_y = split(df_train)
    
    ################################# NN #################################
    
    if load:         
        model_name_rd_nn = f'models/nn/nn_model_{coupling_type}.hdf5'
        nn_model = create_nn_model(validation_X.shape[1]) # vals
        print(f'Loading weights from {model_name_rd_nn}.')
        nn_model.load_weights(model_name_rd_nn)
    else:
        nn_model = nn_train(coupling_type, train_X, train_y, validation_X, validation_y)
    
    nn_val_predict = nn_model.predict(validation_X)

    nn_accuracy = np.log(np.mean(np.abs(validation_y.values - nn_val_predict)))
    print(f'Validation score for {coupling_type} is {nn_accuracy} with NN.\n')
    
    ################################# XGB #################################

    if load:
        model_name_rd_xgb = f'..models/xgb/featurebook_{coupling_type}.joblib.dat'

        xgb_model = xgb.XGBRegressor(base_score = 0.5, booster = 'gbtree', colsample_bylevel = 1,
                                     colsample_bytree = 1, gamma = 0, importance_type = 'gain',
                                     learning_rate = 0.1, max_delta_step = 0, max_depth = 9,
                                     min_child_weight = 1, missing = None, n_estimators = 10000, n_jobs = -1,
                                     nthread = None, objective = 'reg:squarederror', random_state = 101, reg_alpha = 2,
                                     reg_lambda = 0.2, scale_pos_weight = 1, seed = None, silent = False, subsample = 1)

        print(f'Loading weights from {model_name_rd_xgb}.')
        #xgb_model.load_model(model_name_rd_xgb)
        xgb_model= joblib.load(model_name_rd_xgb)
    else:
        xgb_model = xgb_train(coupling_type, train_X, train_y, validation_X, validation_y)   
        
    xgb_val_predict = xgb_model.predict(validation_X)
    
    memory_optimization([train_X, train_y, validation_X])
    gc.collect()
    
    validation_y = validation_y['scalar_coupling_constant'].values

    diff = validation_y - xgb_val_predict
    xgb_accuracy = np.log(np.mean(np.abs(diff)))

    print(f'Validation score for {coupling_type} is {xgb_accuracy} with XGB.\n')
     
    ################################# BLEND #################################
        
    nn_val_predict = np.transpose(nn_val_predict[:,0])
    log_accuracy_list = val_blending_list(nn_val_predict, xgb_val_predict, validation_y)
    print(f'Accuracy_list with alphas for {coupling_type}:\n {log_accuracy_list}')
    
    alpha_i = np.argmin(log_accuracy_list)
    log_accuracy = log_accuracy_list[alpha_i]
    val_score[coupling_type] = (log_accuracy)
    alpha = alpha_i * 0.1
    print(f'Blending with alpha = {alpha}, final accuracy for {coupling_type} = {log_accuracy}.') 
    
    ################################# PREDICT #################################
    
    print('Predicting NN:')
    nn_test_predict = nn_model.predict(df_test)
    nn_test_predict_scaled = alpha * nn_test_predict
    nn_test_predict_scaled = np.transpose(nn_test_predict_scaled[:, 0])
    
    print('Predicting XGB:')
    xgb_test_predict = xgb_model.predict(df_test)
    xgb_test_predict_scaled = (1 - alpha) * xgb_test_predict
    
    test_predict = nn_test_predict_scaled + xgb_test_predict_scaled
    test_prediction[df_test_full['type'] == coupling_type] = test_predict

    memory_optimization([df_test, nn_model, nn_val_predict, xgb_model, xgb_val_predict, nn_test_predict, 
                         nn_test_predict_scaled, xgb_test_predict, xgb_test_predict_scaled, test_predict])
    gc.collect()
    
val_score_total = sum(val_score.values()) / len(val_score.keys())
print(f'Total cv score is {val_score_total}.')

Predicting 1JHN out of ['1JHN', '1JHC', '2JHH', '2JHC', '2JHN', '3JHH', '3JHC', '3JHN'] at 08.13.2019_12.24.33
Train on 39026 samples, validate on 4337 samples
Epoch 1/2000
39026/39026 [==============================] - 15s 384us/sample - loss: 47.0821 - val_loss: 29.6951
Epoch 2/2000
39026/39026 [==============================] - 8s 213us/sample - loss: 46.1115 - val_loss: 34.6080
Epoch 3/2000
39026/39026 [==============================] - 9s 232us/sample - loss: 45.1914 - val_loss: 39.4792
Epoch 4/2000
39026/39026 [==============================] - 9s 221us/sample - loss: 44.2320 - val_loss: 50.6680
Epoch 5/2000
39026/39026 [==============================] - 10s 260us/sample - loss: 43.2961 - val_loss: 39.3412
Epoch 6/2000
39026/39026 [==============================] - 10s 245us/sample - loss: 42.3933 - val_loss: 30.2236
Epoch 7/2000
39026/39026 [==============================] - 9s 240us/sample - loss: 41.4403 - val_loss: 32.7161
Epoch 8/2000
39026/39026 [===========================

Epoch 73/2000
39026/39026 [==============================] - 12s 300us/sample - loss: 9.1526 - val_loss: 8.6008
Epoch 74/2000
39026/39026 [==============================] - 9s 239us/sample - loss: 9.1357 - val_loss: 9.3986
Epoch 75/2000
39026/39026 [==============================] - 10s 252us/sample - loss: 9.2055 - val_loss: 8.9691
Epoch 76/2000
39026/39026 [==============================] - 9s 228us/sample - loss: 9.1553 - val_loss: 8.6877
Epoch 77/2000
39026/39026 [==============================] - 9s 230us/sample - loss: 9.1376 - val_loss: 8.9515
Epoch 78/2000
39026/39026 [==============================] - 9s 231us/sample - loss: 9.1204 - val_loss: 8.9834
Epoch 79/2000
39026/39026 [==============================] - 9s 229us/sample - loss: 9.1440 - val_loss: 8.5056
Epoch 80/2000
39026/39026 [==============================] - 9s 227us/sample - loss: 9.1692 - val_loss: 9.2283
Epoch 81/2000
39026/39026 [==============================] - 9s 231us/sample - loss: 9.1443 - val_loss: 8.5026

Epoch 328/2000
39026/39026 [==============================] - 8s 209us/sample - loss: 8.7660 - val_loss: 7.8468
Epoch 329/2000
39026/39026 [==============================] - 8s 207us/sample - loss: 8.7611 - val_loss: 7.7845
Epoch 330/2000
39026/39026 [==============================] - 8s 207us/sample - loss: 8.7357 - val_loss: 7.7902
Epoch 331/2000
39026/39026 [==============================] - 8s 206us/sample - loss: 8.7276 - val_loss: 7.8091
Epoch 332/2000
39026/39026 [==============================] - 8s 205us/sample - loss: 8.7511 - val_loss: 7.8057
Epoch 333/2000
39026/39026 [==============================] - 8s 208us/sample - loss: 8.7800 - val_loss: 7.7944
Epoch 334/2000
39026/39026 [==============================] - 8s 207us/sample - loss: 8.8144 - val_loss: 7.8273
Epoch 335/2000
39026/39026 [==============================] - 8s 210us/sample - loss: 8.7313 - val_loss: 7.7813
Epoch 336/2000
39026/39026 [==============================] - 8s 212us/sample - loss: 8.7151 - val_loss:

[180]	validation_0-mae:0.394893
[181]	validation_0-mae:0.394737
[182]	validation_0-mae:0.394524
[183]	validation_0-mae:0.394049
[184]	validation_0-mae:0.393686
[185]	validation_0-mae:0.393393
[186]	validation_0-mae:0.39308
[187]	validation_0-mae:0.392782
[188]	validation_0-mae:0.392334
[189]	validation_0-mae:0.391961
[190]	validation_0-mae:0.391646
[191]	validation_0-mae:0.391494
[192]	validation_0-mae:0.390922
[193]	validation_0-mae:0.390658
[194]	validation_0-mae:0.390204
[195]	validation_0-mae:0.389987
[196]	validation_0-mae:0.389661
[197]	validation_0-mae:0.389522
[198]	validation_0-mae:0.389206
[199]	validation_0-mae:0.389037
[200]	validation_0-mae:0.388732
[201]	validation_0-mae:0.388534
[202]	validation_0-mae:0.388201
[203]	validation_0-mae:0.387934
[204]	validation_0-mae:0.387742
[205]	validation_0-mae:0.387429
[206]	validation_0-mae:0.387164
[207]	validation_0-mae:0.387018
[208]	validation_0-mae:0.386857
[209]	validation_0-mae:0.386701
[210]	validation_0-mae:0.38631
[211]	vali

[437]	validation_0-mae:0.358168
[438]	validation_0-mae:0.358166
[439]	validation_0-mae:0.358131
[440]	validation_0-mae:0.358035
[441]	validation_0-mae:0.357996
[442]	validation_0-mae:0.357903
[443]	validation_0-mae:0.357818
[444]	validation_0-mae:0.357756
[445]	validation_0-mae:0.357668
[446]	validation_0-mae:0.357596
[447]	validation_0-mae:0.357583
[448]	validation_0-mae:0.357561
[449]	validation_0-mae:0.357509
[450]	validation_0-mae:0.357442
[451]	validation_0-mae:0.357344
[452]	validation_0-mae:0.357245
[453]	validation_0-mae:0.357189
[454]	validation_0-mae:0.357101
[455]	validation_0-mae:0.357009
[456]	validation_0-mae:0.356949
[457]	validation_0-mae:0.356851
[458]	validation_0-mae:0.356795
[459]	validation_0-mae:0.356722
[460]	validation_0-mae:0.356702
[461]	validation_0-mae:0.356625
[462]	validation_0-mae:0.356638
[463]	validation_0-mae:0.356574
[464]	validation_0-mae:0.356503
[465]	validation_0-mae:0.356472
[466]	validation_0-mae:0.356442
[467]	validation_0-mae:0.356363
[468]	va

[694]	validation_0-mae:0.349733
[695]	validation_0-mae:0.349722
[696]	validation_0-mae:0.349701
[697]	validation_0-mae:0.349656
[698]	validation_0-mae:0.349644
[699]	validation_0-mae:0.349639
[700]	validation_0-mae:0.349611
[701]	validation_0-mae:0.3496
[702]	validation_0-mae:0.349595
[703]	validation_0-mae:0.349585
[704]	validation_0-mae:0.349565
[705]	validation_0-mae:0.349541
[706]	validation_0-mae:0.349526
[707]	validation_0-mae:0.349507
[708]	validation_0-mae:0.349466
[709]	validation_0-mae:0.349435
[710]	validation_0-mae:0.349416
[711]	validation_0-mae:0.349399
[712]	validation_0-mae:0.349373
[713]	validation_0-mae:0.34935
[714]	validation_0-mae:0.349313
[715]	validation_0-mae:0.349264
[716]	validation_0-mae:0.349238
[717]	validation_0-mae:0.349207
[718]	validation_0-mae:0.349212
[719]	validation_0-mae:0.349191
[720]	validation_0-mae:0.34917
[721]	validation_0-mae:0.349163
[722]	validation_0-mae:0.349152
[723]	validation_0-mae:0.349129
[724]	validation_0-mae:0.349105
[725]	valida

[951]	validation_0-mae:0.345857
[952]	validation_0-mae:0.345846
[953]	validation_0-mae:0.345839
[954]	validation_0-mae:0.345834
[955]	validation_0-mae:0.345818
[956]	validation_0-mae:0.345808
[957]	validation_0-mae:0.345789
[958]	validation_0-mae:0.345785
[959]	validation_0-mae:0.345782
[960]	validation_0-mae:0.345774
[961]	validation_0-mae:0.345778
[962]	validation_0-mae:0.345766
[963]	validation_0-mae:0.345763
[964]	validation_0-mae:0.345746
[965]	validation_0-mae:0.345744
[966]	validation_0-mae:0.345718
[967]	validation_0-mae:0.345698
[968]	validation_0-mae:0.345671
[969]	validation_0-mae:0.345669
[970]	validation_0-mae:0.34565
[971]	validation_0-mae:0.345645
[972]	validation_0-mae:0.345638
[973]	validation_0-mae:0.345629
[974]	validation_0-mae:0.345629
[975]	validation_0-mae:0.345618
[976]	validation_0-mae:0.345624
[977]	validation_0-mae:0.345614
[978]	validation_0-mae:0.345605
[979]	validation_0-mae:0.345604
[980]	validation_0-mae:0.345607
[981]	validation_0-mae:0.345596
[982]	val

[1202]	validation_0-mae:0.343906
[1203]	validation_0-mae:0.343905
[1204]	validation_0-mae:0.343899
[1205]	validation_0-mae:0.34389
[1206]	validation_0-mae:0.343888
[1207]	validation_0-mae:0.343878
[1208]	validation_0-mae:0.343873
[1209]	validation_0-mae:0.343867
[1210]	validation_0-mae:0.343864
[1211]	validation_0-mae:0.343864
[1212]	validation_0-mae:0.343857
[1213]	validation_0-mae:0.343855
[1214]	validation_0-mae:0.343841
[1215]	validation_0-mae:0.343839
[1216]	validation_0-mae:0.343836
[1217]	validation_0-mae:0.343837
[1218]	validation_0-mae:0.343825
[1219]	validation_0-mae:0.34382
[1220]	validation_0-mae:0.343802
[1221]	validation_0-mae:0.343796
[1222]	validation_0-mae:0.343788
[1223]	validation_0-mae:0.343793
[1224]	validation_0-mae:0.343789
[1225]	validation_0-mae:0.34378
[1226]	validation_0-mae:0.34377
[1227]	validation_0-mae:0.34376
[1228]	validation_0-mae:0.343754
[1229]	validation_0-mae:0.343754
[1230]	validation_0-mae:0.343757
[1231]	validation_0-mae:0.34375
[1232]	validatio

[1452]	validation_0-mae:0.342825
[1453]	validation_0-mae:0.342825
[1454]	validation_0-mae:0.342825
[1455]	validation_0-mae:0.342825
[1456]	validation_0-mae:0.342825
[1457]	validation_0-mae:0.342825
[1458]	validation_0-mae:0.342825
[1459]	validation_0-mae:0.342825
[1460]	validation_0-mae:0.342825
[1461]	validation_0-mae:0.342825
[1462]	validation_0-mae:0.342825
[1463]	validation_0-mae:0.342825
[1464]	validation_0-mae:0.342825
[1465]	validation_0-mae:0.342825
[1466]	validation_0-mae:0.342825
[1467]	validation_0-mae:0.342825
[1468]	validation_0-mae:0.342825
[1469]	validation_0-mae:0.342825
[1470]	validation_0-mae:0.342825
[1471]	validation_0-mae:0.342825
[1472]	validation_0-mae:0.342825
[1473]	validation_0-mae:0.342825
[1474]	validation_0-mae:0.342825
[1475]	validation_0-mae:0.342825
[1476]	validation_0-mae:0.342825
[1477]	validation_0-mae:0.342825
[1478]	validation_0-mae:0.342825
[1479]	validation_0-mae:0.342825
[1480]	validation_0-mae:0.342825
[1481]	validation_0-mae:0.342825
[1482]	val

KeyboardInterrupt: 

## Make submission csv

In [ ]:
def submit(predictions):
    submit = pd.read_csv('submissions/submission_best.csv')  
    submit['scalar_coupling_constant'] = predictions
    submit.to_csv('submissions/submission_blended_full.csv', index = False)

In [ ]:
submit(test_prediction)

time = datetime.now(pytz.timezone('Europe/Oslo')).strftime('%m.%d.%Y_%H.%M.%S')
print(f'Notebook EoF reached at {time} and submission saved.')